In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\4\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\4\test'
checkpoint_path = r'C:\Users\24306\Desktop\EfficientNetV2B0\4\EfficientNetV2B0.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[66.0, 95.0, 66.0, 40.0, 4.0, 13.0, 23.0, 93.0, 13.0, 45.0, 16.0, 52.0, 66.0, 95.0, 55.0, 55.0, 96.0, 47.0, 66.0, 32.0, 23.0, 100.0, 10.0, 10.0, 37.0, 37.0, 4.0, 40.0, 47.0, 7.0, 10.0, 95.0, 84.0, 84.0, 35.0, 13.0, 93.0, 37.0, 32.0, 80.0, 93.0, 20.0, 10.0, 86.0, 80.0, 80.0, 45.0, 100.0, 37.0, 80.0, 86.0, 40.0, 32.0, 52.0, 95.0, 95.0, 86.0, 4.0, 40.0, 52.0, 80.0, 37.0, 80.0, 96.0, 47.0, 84.0, 86.0, 23.0, 80.0, 39.0, 86.0, 13.0, 16.0, 86.0, 40.0, 13.0, 7.0, 37.0, 40.0, 16.0, 95.0, 30.0, 10.0, 93.0, 37.0, 4.0, 10.0, 47.0, 100.0, 84.0, 100.0, 40.0, 84.0, 40.0, 37.0, 37.0, 30.0, 100.0, 32.0, 30.0, 55.0, 23.0, 55.0, 96.0, 10.0, 7.0, 80.0, 40.0, 96.0, 32.0, 30.0, 45.0, 13.0, 10.0, 96.0, 86.0, 13.0, 23.0, 47.0, 66.0, 10.0, 47.0, 66.0, 7.0, 80.0, 45.0, 16.0, 30.0, 95.0, 100.0, 16.0, 35.0, 23.0, 23.0, 52.0, 47.0, 37.0, 40.0, 37.0, 40.0, 86.0, 84.0, 30.0, 47.0, 16.0, 7.0, 37.0, 23.0, 47.0, 32.0, 23.0, 37.0, 84.0, 47.0, 95.0, 30.0, 100.0, 66.0, 66.0, 32.0, 95.0, 37.0, 37.0, 100.0, 16.0, 47.0, 39.0

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
EfficientNetV2B0 = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
EfficientNetV2B0.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = EfficientNetV2B0(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-b0 (Function  (None, 7, 7, 1280)       5919312   
 al)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 1281      
                                                                 
Total params: 5,920,593
Trainable params: 5,859,985
Non-trainable params: 60,608
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('EfficientNetV2B0_loss_4.txt',history.history['loss'])
#要改
np.savetxt('EfficientNetV2B0_valloss_4.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Thu Apr  6 08:47:15 2023
Epoch 1/300
592/592 [==============================] - ETA: 0s - loss: 247.2986

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2B0\4\EfficientNetV2B0.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2B0\4\EfficientNetV2B0.ckpt\assets


592/592 [==============================] - 116s 163ms/step - loss: 247.2986 - val_loss: 837.1254
Epoch 2/300
592/592 [==============================] - ETA: 0s - loss: 9.1798

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2B0\4\EfficientNetV2B0.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2B0\4\EfficientNetV2B0.ckpt\assets


592/592 [==============================] - 170s 287ms/step - loss: 0.2301 - val_loss: 0.0368
Epoch 261/300
592/592 [==============================] - 130s 219ms/step - loss: 0.1311 - val_loss: 0.0470
Epoch 262/300
592/592 [==============================] - 129s 217ms/step - loss: 0.1302 - val_loss: 0.0384
Epoch 263/300
592/592 [==============================] - 128s 217ms/step - loss: 0.2095 - val_loss: 0.0454
Epoch 264/300
592/592 [==============================] - 127s 215ms/step - loss: 0.1335 - val_loss: 0.0579
Epoch 265/300
592/592 [==============================] - 127s 214ms/step - loss: 0.1229 - val_loss: 0.0422
Epoch 266/300
592/592 [==============================] - 127s 215ms/step - loss: 0.1347 - val_loss: 0.0506
Epoch 267/300
592/592 [==============================] - 129s 218ms/step - loss: 0.1432 - val_loss: 0.0416
Epoch 268/300
592/592 [==============================] - 129s 218ms/step - loss: 0.1262 - val_loss: 0.0399
Epoch 269/300
592/592 [============================

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2B0\4\EfficientNetV2B0.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2B0\4\EfficientNetV2B0.ckpt\assets


592/592 [==============================] - 173s 292ms/step - loss: 0.1508 - val_loss: 0.0362
Epoch 281/300
592/592 [==============================] - 129s 218ms/step - loss: 0.1230 - val_loss: 0.0412
Epoch 282/300
592/592 [==============================] - 129s 218ms/step - loss: 0.2089 - val_loss: 0.0428
Epoch 283/300
592/592 [==============================] - 129s 218ms/step - loss: 0.1685 - val_loss: 0.0556
Epoch 284/300
592/592 [==============================] - 128s 216ms/step - loss: 0.1161 - val_loss: 0.0457
Epoch 285/300
592/592 [==============================] - 126s 213ms/step - loss: 0.1421 - val_loss: 0.0439
Epoch 286/300
592/592 [==============================] - 128s 216ms/step - loss: 0.1378 - val_loss: 0.0461
Epoch 287/300
592/592 [==============================] - 127s 215ms/step - loss: 0.1169 - val_loss: 0.0400
Epoch 288/300
592/592 [==============================] - 125s 211ms/step - loss: 0.1240 - val_loss: 0.0497
Epoch 289/300
592/592 [============================